# Etude d'un système de spheres dures par simulation moléculaire

In [ ]:
import numpy,pandas
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import os

## Equation d'état à basse densité -- Deuxième coefficient du viriel

### Equation d'état à partir d'une longue simulation

In [ ]:
#collisionData = pandas.read_csv('data/collisionData_3_3_3_0.00022_1_100000_0_0.02_5_6.dat')
collisionData = pandas.read_csv('data/collisionData3.csv')
[t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

print (t[-1])

In [ ]:
N = 108
T = 1
Delta_t = 100

p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression*beta/densité
for i in range(len(t)):
    p[int((t[i]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[i]
    
p = p[:-1] # dernier bin incomplet

In [ ]:
pyplot.figure(figsize=(8,5));
pyplot.plot(p, '.');
pyplot.title("p avec petit Delta_t");
pyplot.xlabel('t/Delta_t');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

In [ ]:
p_moy = numpy.sum(p)/len(p)
print (p_moy)

#### Divise en deux moitiés

In [ ]:
M = int(len(t)/2)

t1 = t[:M]
t2 = t[M:]
vDotr1 = vDotr[:M]
vDotr2 = vDotr[M:]

N = 108
T = 1
Delta_t = 100

p1 = numpy.ones(int((t1[-1]-t1[0])/Delta_t)+1) # pression*beta/densité
for i in range(len(t1)):
    p1[int((t1[i]-t1[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr1[i]
p2 = p1[:-1]

p2 = numpy.ones(int((t2[-1]-t2[0])/Delta_t)+1) # pression*beta/densité
for i in range(len(t2)):
    p2[int((t2[i]-t2[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr2[i]
p2 = p2[:-1]
    
p_moy1 = numpy.sum(p1)/len(p1)
print (p_moy1)
p_moy2 = numpy.sum(p2)/len(p2)
print (p_moy2)

pyplot.figure(figsize=(8,5));
pyplot.plot(p1, '.');
pyplot.title("p1 avec petit Delta_t");
pyplot.xlabel('t/Delta_t');
pyplot.ylabel('p1');
pyplot.grid();
pyplot.show()

pyplot.figure(figsize=(8,5));
pyplot.plot(p2, '.');
pyplot.title("p2 avec petit Delta_t");
pyplot.xlabel('t/Delta_t');
pyplot.ylabel('p2');
pyplot.grid();
pyplot.show()

### Calcul du temps de corrélation

La détermination du temps de corrélation est importante pour connaître le durée $\Delta t$ à utiliser pour calculer la moyenne et l'erreur sur la pression du système. En effet, les valeurs de la pression calculées sur un intervalle $\Delta t$ trop petit sont corrélées entre elles ce qui complique le calcul de l'erreur statistique sur la pression moyenne. Le temps de corrélation $\tau$ d'une observable $A$ est défini à partir de la fonction de corrélation  $c_{AA}$:

$$ \tau = \frac 12 \sum_{n=-\infty}^{+\infty} \frac{c_{AA}(n)}{c_{AA}(0)} $$
$$ c_{AA}(k) = \langle A_n A_{n+k} \rangle - \langle A_n \rangle ^2 $$

L'observable en question est la "pression réduite" que l'on a défini comme:

$$ p_n = \frac{\beta P_n}{n} = 1 + \frac{1}{3 N T \Delta t} \sum v_{ij}\cdot r_{ij} \qquad \text{somme sur les collisions durant [}n\Delta t\text{ , } (n+1)\Delta t \text{[} $$

In [ ]:
def c_pp(k,p):
    
    N = len(p)
    
    if k>=0:
        avg1 = numpy.sum(p[0:N-k]*p[k:N]) / (N-k) # < p_n*p_{n+k} >
        avg2 = numpy.sum(p[0:N-k]) / (N-k) # < p_n >
    elif k<0:
        avg1 = numpy.sum(p[-k:N]*p[0:N+k]) / (N+k) # < p_n*p_{n+k} >
        avg2 = numpy.sum(p[-k:N]) / (N+k) # < p_n >
    
    return avg1-avg2**2

In [ ]:
N = 108
T = 1
Delta_t = 100

p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression réduite
for i in range(len(t)):
    p[int((t[i]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[i]
    
p = p[:-1] # dernier bin incomplet

On vérifie que $c_{pp}(0)$ est bien la variance du vecteur des pressions réduites.

In [ ]:
print( c_pp(0,p) )
print( numpy.std(p)**2 )

<span style="color:red"> La fonction de corrélation semble beaucoup trop grande. Il me semble qu'elle est censée décroître quand $k$ s'éloigne de $0$. Est-ce donc une erreur ou alors le système est corrélé sur des temps très longs ? </span>

In [ ]:
for i in range(int(len(p)/2)):
    print( "c_pp({:d},p) = {:.3f} E-6".format(i,c_pp(i,p)*1000000) )

### Execution de plusieurs simulations pour avoir des données indépendantes

On exécute les commandes linux servant à effectuer les simulations avec le module "os".

In [ ]:
NSim = 20 # nombre de simulations

for i in range(NSim):
    os.system('./simulationO3 3 3 3 0.00022 1 1000 0.05 5 6 0')
    os.system("mv data/collisionData.csv data/collisionData{:d}.csv".format(i))
    os.system("mv data/infoSimulation.csv data/infoSimulation{:d}.csv".format(i))

In [ ]:
N = 108
T = 1
Delta_t = 100

p_sim = numpy.zeros(NSim)

for i in range(NSim):
    
    collisionData = pandas.read_csv("data/collisionData{:d}.csv".format(i))
    [t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

    p = numpy.ones(int((t[-1]-t[0])/Delta_t)+1) # pression réduite
    for j in range(len(t)):
        p[int((t[j]-t[0])/Delta_t)] += 1/(N*3*T*Delta_t) * vDotr[j]

    p = p[:-1] # dernier bin incomplet

    p_sim[i] = numpy.sum(p)/len(p)

print (p_sim)

pyplot.figure(figsize=(8,5));
pyplot.plot(p_sim, '.');
pyplot.title("p pour plusieurs simulations");
pyplot.xlabel('t/Delta_t');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()